# 133 - Clone Graph

# Base Version

In [3]:
# Definition for a Node (provided by LeetCode)
class Node:
    def __init__(self, val=0, neighbors=None):
        self.val = val
        self.neighbors = neighbors if neighbors is not None else []

from collections import deque

# ----- Your minimal BFS clone -----
def cloneGraph(node: 'Node') -> 'Node':
    if not node:
        return None
    old_to_new = {node: Node(node.val)}
    q = deque([node])
    while q:
        cur = q.popleft()
        for nei in cur.neighbors:
            if nei not in old_to_new:
                old_to_new[nei] = Node(nei.val)
                q.append(nei)
            old_to_new[cur].neighbors.append(old_to_new[nei])
    return old_to_new[node]

# ----- Helpers: build and serialize -----
def build_graph(adjList):
    """Builds a connected undirected graph from LeetCode-style adjList and returns node with val=1."""
    if not adjList:
        return None
    n = len(adjList)
    nodes = {i: Node(i) for i in range(1, n+1)}
    for i in range(1, n+1):
        nodes[i].neighbors = [nodes[v] for v in adjList[i-1]]
    return nodes[1]  # LeetCode passes the node with val=1 as the entry

def serialize_graph(start):
    """Serializes a connected graph back to LeetCode-style adjList (neighbors listed by node.val)."""
    if not start:
        return []
    # BFS to discover all nodes and map by value
    q = deque([start])
    seen = set([start])
    by_val = {start.val: start}
    while q:
        cur = q.popleft()
        for nei in cur.neighbors:
            if nei not in seen:
                seen.add(nei)
                q.append(nei)
                by_val[nei.val] = nei
    # Create adj list ordered by 1..max_val seen
    n = max(by_val.keys())
    adj = [[] for _ in range(n)]
    for v in range(1, n+1):
        if v in by_val:
            adj[v-1] = [nei.val for nei in by_val[v].neighbors]
        else:
            adj[v-1] = []
    return adj

# ----- Test with the sample -----
adjList = [[2,4],[1,3],[2,4],[1,3]]
orig_start = build_graph(adjList)
cloned_start = cloneGraph(orig_start)
out = serialize_graph(cloned_start)

print("Input: ", adjList)
print("Output:", out)

Input:  [[2, 4], [1, 3], [2, 4], [1, 3]]
Output: [[2, 4], [1, 3], [2, 4], [1, 3]]


# Verbose Version

In [4]:
# LeetCode's provided Node definition
class Node:
    def __init__(self, val=0, neighbors=None):
        self.val = val
        self.neighbors = neighbors if neighbors is not None else []

from collections import deque

def cloneGraph_verbose_bfs(node: 'Node') -> 'Node':
    if not node:
        print("Input node is None → return None")
        return None

    # Map original node object -> cloned node object
    old_to_new = {}

    # Create the clone of the start node
    clone_start = Node(node.val)
    old_to_new[node] = clone_start

    q = deque([node])

    print("=== START BFS CLONE ===")
    print(f"Initialize: clone node {node.val}, queue = [{node.val}]")
    print_state(old_to_new, q)

    # Standard BFS
    step = 0
    while q:
        step += 1
        cur = q.popleft()
        print(f"\n[Step {step}] Dequeue original node {cur.val}")

        # For each neighbor of the current original node...
        for nei in cur.neighbors:
            # If neighbor hasn't been cloned yet, create clone and enqueue
            if nei not in old_to_new:
                old_to_new[nei] = Node(nei.val)
                q.append(nei)
                print(f"  - Neighbor {nei.val} not seen → clone it and enqueue")
            else:
                print(f"  - Neighbor {nei.val} already cloned")

            # Wire the edge on the clone side:
            # add cloned neighbor to cloned current node's neighbors
            old_to_new[cur].neighbors.append(old_to_new[nei])
            print(f"    → Add edge (clone {cur.val}) -- (clone {nei.val})")

        print_state(old_to_new, q)

    print("\n=== FINISH BFS CLONE ===")
    return clone_start

def print_state(old_to_new, q):
    # Helper to print mapping and queue nicely
    # Map summary
    mapping_summary = []
    for orig, clone in old_to_new.items():
        mapping_summary.append(f"{orig.val}->{clone.val}")
    mapping_summary.sort()

    # Queue summary
    queue_vals = [n.val for n in q]

    print("Current mapping old_to_new:", mapping_summary)
    print("Current queue:", queue_vals)

# Build the 4-node square graph: 1-2-3-4-1
n1, n2, n3, n4 = Node(1), Node(2), Node(3), Node(4)
n1.neighbors = [n2, n4]
n2.neighbors = [n1, n3]
n3.neighbors = [n2, n4]
n4.neighbors = [n1, n3]

# Test the verbose clone
clone = cloneGraph_verbose_bfs(n1)


=== START BFS CLONE ===
Initialize: clone node 1, queue = [1]
Current mapping old_to_new: ['1->1']
Current queue: [1]

[Step 1] Dequeue original node 1
  - Neighbor 2 not seen → clone it and enqueue
    → Add edge (clone 1) -- (clone 2)
  - Neighbor 4 not seen → clone it and enqueue
    → Add edge (clone 1) -- (clone 4)
Current mapping old_to_new: ['1->1', '2->2', '4->4']
Current queue: [2, 4]

[Step 2] Dequeue original node 2
  - Neighbor 1 already cloned
    → Add edge (clone 2) -- (clone 1)
  - Neighbor 3 not seen → clone it and enqueue
    → Add edge (clone 2) -- (clone 3)
Current mapping old_to_new: ['1->1', '2->2', '3->3', '4->4']
Current queue: [4, 3]

[Step 3] Dequeue original node 4
  - Neighbor 1 already cloned
    → Add edge (clone 4) -- (clone 1)
  - Neighbor 3 already cloned
    → Add edge (clone 4) -- (clone 3)
Current mapping old_to_new: ['1->1', '2->2', '3->3', '4->4']
Current queue: [3]

[Step 4] Dequeue original node 3
  - Neighbor 2 already cloned
    → Add edge (clo